In [42]:
import pandas as pd
import geopandas as gpd
import numpy as np

import shapely
from shapely.geometry import Point

import tqdm

### 교통량 데이터

In [20]:
traffic = pd.read_excel('../데이터/입지분석/전처리_완속/교통량/추정교통량.xlsx')
traffic.head()

,LINK ID,ITS LINK ID,도로등급,도로명,권역,연장(km),차선수,전일,1시,2시,...,15시.3,16시.3,17시.3,18시.3,19시.3,20시.3,21시.3,22시.3,23시.3,24시.3
0,1013669,1160001701,일반국도,경인로,서울특별시 구로구,0.1575,3,50718,984,710,...,446,448,466,489,470,417,368,344,300,230
1,1013670,1160001801,일반국도,경인로,서울특별시 구로구,0.1575,3,6735,139,93,...,3,3,4,4,4,3,3,3,3,3
2,1013671,1160055000,특별광역시도,경인로,서울특별시 구로구,0.0820,1,28,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1013672,1160001302,일반국도,경인로,서울특별시 구로구,0.0947,3,6707,140,95,...,4,4,4,4,4,4,4,4,4,3
4,1013673,1160001402,일반국도,경인로,서울특별시 구로구,0.0947,3,31338,614,443,...,278,279,291,305,291,257,226,211,183,138


In [21]:
traffic = traffic[(traffic['도로등급'] != '도시고속도로') & (traffic['도로등급'] != '고속도로')]

traffic = traffic[['LINK ID', '전일']]

traffic.head()

,LINK ID,전일
0,1013669,50718
1,1013670,6735
2,1013671,28
3,1013672,6707
4,1013673,31338


In [16]:
road = gpd.read_file('../데이터/입지분석/전처리_완속/교통량/2018년_서울특별시_주요도로망Level5.5.shp', encoding = 'cp949')
road.head()

,k_link_id,fnode_id,tnode_id,road_name,road_no,road_rank,link_type,lane,road_info,sido_id,sigungu_id,emd_id,geometry
0,1013669,201397,201403,경인로,46,103,32768,3,None,11000,11170,1117072,"LINESTRING (186584.769 544304.016, 186741.276 ..."
1,1013670,201403,201397,경인로,46,103,32768,3,None,11000,11170,1117072,"LINESTRING (186741.067 544323.362, 186584.560 ..."
2,1013671,201397,201941,경인로,0,104,8,1,None,11000,11170,1117072,"LINESTRING (186584.904 544306.017, 186573.594 ..."
3,1013672,201397,201425,경인로,46,103,32768,3,None,11000,11170,1117072,"LINESTRING (186584.591 544306.014, 186562.075 ..."
4,1013673,201425,201397,경인로,46,103,32768,3,None,11000,11170,1117072,"LINESTRING (186497.471 544272.964, 186531.639 ..."


In [17]:
road = road[['k_link_id', 'geometry']]
road.head()

,k_link_id,geometry
0,1013669,"LINESTRING (186584.769 544304.016, 186741.276 ..."
1,1013670,"LINESTRING (186741.067 544323.362, 186584.560 ..."
2,1013671,"LINESTRING (186584.904 544306.017, 186573.594 ..."
3,1013672,"LINESTRING (186584.591 544306.014, 186562.075 ..."
4,1013673,"LINESTRING (186497.471 544272.964, 186531.639 ..."


In [18]:
road.columns = ['LINK ID', 'geometry']
road = road.set_index('LINK ID')
road.head()

,geometry
LINK ID,
1013669,"LINESTRING (186584.769 544304.016, 186741.276 ..."
1013670,"LINESTRING (186741.067 544323.362, 186584.560 ..."
1013671,"LINESTRING (186584.904 544306.017, 186573.594 ..."
1013672,"LINESTRING (186584.591 544306.014, 186562.075 ..."
1013673,"LINESTRING (186497.471 544272.964, 186531.639 ..."


In [24]:
road.index = road.index.astype(int)
road.index

Int64Index([1013669, 1013670, 1013671, 1013672, 1013673, 1013674, 1013675,
            1013676, 1013677, 1013678,
            ...
            8572048, 8572049, 8572073, 8572074, 8572145, 8572146, 8572159,
            8572160, 8572172, 8572173],
           dtype='int64', name='LINK ID', length=8133)

In [27]:
traffic = road.join(traffic.set_index('LINK ID'), how = 'inner')

traffic

,geometry,전일
LINK ID,,
1013669,"LINESTRING (186584.769 544304.016, 186741.276 ...",50718
1013670,"LINESTRING (186741.067 544323.362, 186584.560 ...",6735
1013671,"LINESTRING (186584.904 544306.017, 186573.594 ...",28
1013672,"LINESTRING (186584.591 544306.014, 186562.075 ...",6707
1013673,"LINESTRING (186497.471 544272.964, 186531.639 ...",31338
...,...,...
8572146,"LINESTRING (203261.719 540071.331, 203309.438 ...",15590
8572159,"LINESTRING (197492.147 550962.243, 197493.022 ...",32837
8572160,"LINESTRING (197541.089 550867.883, 197525.684 ...",23358


In [28]:
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret

In [29]:
traffic = traffic.to_crs(epsg = 4326)
traffic.head()

,geometry,전일
LINK ID,,
1013669,"LINESTRING (126.84829 37.49810, 126.85006 37.4...",50718
1013670,"LINESTRING (126.85006 37.49828, 126.84829 37.4...",6735
1013671,"LINESTRING (126.84829 37.49812, 126.84817 37.4...",28
1013672,"LINESTRING (126.84829 37.49812, 126.84803 37.4...",6707
1013673,"LINESTRING (126.84730 37.49782, 126.84769 37.4...",31338


In [31]:
traffic.columns = ['geometry', '교통량']

In [30]:
# grid 정보 불러오기
grid = gpd.read_file('../데이터/입지분석/전기차충전소/시범 전기차충전소(시군구격자) 접근성.shp', encoding = 'utf8')

grid = grid[grid['sido_nm_k'] == '서울특별시']
grid = grid[['gid', 'geometry']]

grid.head()

,gid,geometry
0,다사55a53b,"POLYGON ((955000.000 1953500.000, 955000.000 1..."
1,다사53a53a,"POLYGON ((953000.000 1953000.000, 953000.000 1..."
2,다사52b53a,"POLYGON ((952500.000 1953000.000, 952500.000 1..."
3,다사53a57b,"POLYGON ((953000.000 1957500.000, 953000.000 1..."
4,다사53a54b,"POLYGON ((953000.000 1954500.000, 953000.000 1..."


In [79]:
import tqdm

traffic_sum = grid.copy()
traffic_sum = traffic_sum.to_crs(epsg = 4326)

traffic_sum['교통량'] = 0

for poly_index in tqdm.tqdm_notebook(traffic_sum.index):
    for line_index in traffic.index:
        if traffic_sum.at[poly_index, 'geometry'].intersects(traffic.at[line_index, 'geometry']):
            traffic_sum.at[poly_index, '교통량'] += float(traffic.at[line_index, '교통량'])

C:\Users\ms964\AppData\Local\Temp\ipykernel_28428\3468800765.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for poly_index in tqdm.tqdm_notebook(traffic_sum.index):


  0%|          | 0/2420 [00:00<?, ?it/s]

In [81]:
traffic_sum

,gid,geometry,교통량
0,다사55a53b,"POLYGON ((126.99035 37.57979, 126.99032 37.584...",0
1,다사53a53a,"POLYGON ((126.96773 37.57518, 126.96770 37.579...",293423
2,다사52b53a,"POLYGON ((126.96207 37.57515, 126.96204 37.579...",3638
3,다사53a57b,"POLYGON ((126.96744 37.61574, 126.96741 37.620...",0
4,다사53a54b,"POLYGON ((126.96764 37.58870, 126.96761 37.593...",46138
...,...,...,...
2415,다사70b52a,"POLYGON ((127.16595 37.56689, 127.16593 37.571...",21616
2416,다사67b48a,"POLYGON ((127.13216 37.53074, 127.13214 37.535...",138760
2417,다사68a47a,"POLYGON ((127.13786 37.52174, 127.13784 37.526...",0
2418,다사68a51a,"POLYGON ((127.13769 37.55780, 127.13767 37.562...",67453


In [82]:
traffic_sum.set_index('gid')['교통량'].to_excel('../데이터/입지분석/전처리_완속/교통량/누적교통량.xlsx')

### 주차장 개수
- [공공데이터포털](https://www.data.go.kr/data/15012896/standard.do)에서 추출

In [34]:
parking = pd.read_csv('../데이터/입지분석/전처리_급속/주차장/전국주차장정보표준데이터.csv', encoding='cp949')
parking = parking[['주차장구분', '주차장유형', '소재지지번주소', '주차구획수', '요금정보', '위도', '경도']]
parking.head()

,주차장구분,주차장유형,소재지지번주소,주차구획수,요금정보,위도,경도
0,공영,노외,대전광역시 유성구 신성동 211-2,70,무료,36.388520,127.348863
1,공영,노외,대전광역시 유성구 덕명동 544,30,무료,36.356774,127.310327
2,공영,노외,대전광역시 유성구 구암동 657,19,무료,36.360501,127.308142
3,공영,노외,대전광역시 유성구 구암동 677,17,무료,36.358604,127.309319
4,공영,노외,대전광역시 유성구 덕명동 544,29,무료,36.356774,127.310327


In [35]:
parking['시'] = parking['소재지지번주소'].str.split(' ').str[0]

parking = parking[parking['시'] == '서울특별시']
parking.head()

,주차장구분,주차장유형,소재지지번주소,주차구획수,요금정보,위도,경도,시
64,공영,노외,서울특별시 강서구 화곡동 890,248,유료,37.529059,126.848899,서울특별시
65,공영,노외,서울특별시 강서구 화곡동 662-15,500,유료,37.537235,126.837038,서울특별시
66,공영,노외,서울특별시 강서구 화곡동56-127,97,유료,37.539703,126.849774,서울특별시
67,공영,노외,서울특별시 강서구 화곡동424-94,98,유료,37.531422,126.839875,서울특별시
68,공영,노외,서울특별시 강서구 화곡동 410-60,53,유료,37.538512,126.848839,서울특별시


In [38]:
from shapely.geometry import Point

parking['geometry'] = 0

for index in parking.index:
    lat = float(parking.loc[index, '위도'])
    lon = float(parking.loc[index, '경도'])
    parking.loc[index, 'geometry'] = Point(lon, lat)


In [41]:
parking = parking[['주차구획수', 'geometry']]

In [90]:
import tqdm
import numpy as np

parking_sum = grid.copy()

parking_sum = parking_sum.to_crs(epsg = 4326)

parking_sum['주차장수'] = 0
parking_sum['주차가능대수'] = 0

for polygon in tqdm.tqdm_notebook(parking_sum.index):
    for point in parking.index:
        if parking_sum.at[polygon, 'geometry'].intersects(parking.at[point, 'geometry']):
            try:
                parking_sum.at[polygon, '주차장수'] += 1
                parking_sum.at[polygon, '주차가능대수'] += int(parking.at[point, '주차구획수'])
            except TypeError:
                parking_sum.at[polygon, '주차가능대수'] += 47.500000

C:\Users\ms964\AppData\Local\Temp\ipykernel_28428\4025284445.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for polygon in tqdm.tqdm_notebook(parking_sum.index):


  0%|          | 0/2420 [00:00<?, ?it/s]

In [95]:
parking_sum.set_index('gid')[['주차장수', '주차가능대수']].to_excel('../데이터/입지분석/전처리_급속/주차장/주차장수.xlsx')

In [52]:
test = pd.read_excel('../데이터/입지분석/주차장/주차장수_주차가능대수.xlsx')
test

,gid,주차장수,주차가능대수
0,다사53a53a,0,0.0
1,다사55a52a,3,242.0
2,다사55b53a,0,0.0
3,다사56a53b,0,0.0
4,다사56a52b,0,0.0
...,...,...,...
402,다사68b50b,0,0.0
403,다사68a48b,0,0.0
404,다사68b49b,0,0.0
405,다사67b48b,1,189.0


### 급속충전소개수

In [100]:
charger = pd.read_excel('../데이터/입지분석/전처리_급속/충전소/서울시_충전소_좌표.xlsx')

In [102]:
for index in tqdm.tqdm_notebook(charger.index):
    charger['geometry'] = charger.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)

C:\Users\ms964\AppData\Local\Temp\ipykernel_28428\4273006936.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm.tqdm_notebook(charger.index):


  0%|          | 0/4159 [00:00<?, ?it/s]

c:\Users\ms964\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\ms964\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\ms964\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\ms964\AppData\Local\Programs\Python\Python310\lib\site-packag

In [103]:
charger_num = grid.copy()

fast_charger = charger[charger['충전기타입'] != 'AC완속']

In [106]:
charger_num = charger_num.to_crs(epsg = 4326)
charger_num['충전소개수'] = 0

for poly_index in tqdm.tqdm_notebook(charger_num.index):
    for point_index in fast_charger.index:
        if charger_num.at[poly_index, 'geometry'].intersects(fast_charger.at[point_index, 'geometry']):
            charger_num.at[poly_index, '충전소개수'] += 1

C:\Users\ms964\AppData\Local\Temp\ipykernel_28428\3490033370.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for poly_index in tqdm.tqdm_notebook(charger_num.index):


  0%|          | 0/2420 [00:00<?, ?it/s]

In [111]:
charger_num.set_index('gid')['충전소개수'].to_excel('../데이터/입지분석/전처리_급속/충전소/급속충전소개수.xlsx')